In [271]:
import os
import numpy as np
import matplotlib.pyplot as plt
import eplDataset as eds
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
from torchvision import transforms
from torch.utils.data import DataLoader

## Load Training Data

In [272]:
batch_size = 12
# data Transform
trans = transforms.Compose([
                            transforms.ToTensor(),
#                             transforms.Normalize([0.5],[0.5]),
                           ])

train_data = eds.EplDataset(train=True,transform=trans)
test_data = eds.EplDataset(train=False,transform=trans)

train_loader = DataLoader(train_data, batch_size = batch_size, shuffle=True)
test_loader = DataLoader(test_data,batch_size = batch_size)

## MLP Model

In [273]:
class MLP(nn.Module):
    def __init__(self, hidden_units = [512,256,128,64,32]):
        super().__init__()
        
        self.in_dim = 22 # MNIST
        self.out_dim = 3
        
        self.l_layers = []
        self.l_layers.append(nn.Linear(self.in_dim,hidden_units[0]))
        for i in range(len(hidden_units)-1) :
            self.l_layers.append(nn.Linear(hidden_units[i],hidden_units[i+1]))
        self.l_layers.append(nn.Linear(hidden_units[-1],self.out_dim))
        
        self.l_layers = nn.ModuleList(self.l_layers)
        
        self.relu = nn.ReLU()
        self.log_softmax = nn.LogSoftmax()
        
    def forward(self,x):
        a = x.view(-1,self.in_dim)
        output_list = []
        
        for i in range(len(self.l_layers)) :
            z = self.l_layers[i](a)
            output_list.append(z)
            if i != len(self.l_layers) -1 :
                a = self.relu(z)
            else : 
                out = z
                a = self.log_softmax(z)
            output_list.append(a)
                        
        return out, output_list

In [274]:
model = MLP()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.0001,weight_decay = 1e-5)

## Train

In [279]:
for epoch in range(500):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs: data is a list of [inputs, labels]
        inputs, labels = data
        #zero the parameter gradients
        optimizer.zero_grad()
        
        #forward + backward + optimize
        outputs,_ = model(inputs)
        labels = labels.type(torch.LongTensor)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
        if (i+1) % 100 == 0:
            print("[%d, %5d] loss : %f" %
                 (epoch+1, i+1, running_loss/2000))
            running_loss = 0

print("Finished Training")

<ipython-input-273-a89117153749>:30: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  a = self.log_softmax(z)


[1,   100] loss : 0.041239
[1,   200] loss : 0.042485
[2,   100] loss : 0.041827
[2,   200] loss : 0.041271
[3,   100] loss : 0.041520
[3,   200] loss : 0.041662
[4,   100] loss : 0.041644
[4,   200] loss : 0.042238
[5,   100] loss : 0.041451
[5,   200] loss : 0.041249
[6,   100] loss : 0.043085
[6,   200] loss : 0.041777
[7,   100] loss : 0.041818
[7,   200] loss : 0.043350
[8,   100] loss : 0.042308
[8,   200] loss : 0.041583
[9,   100] loss : 0.041793
[9,   200] loss : 0.040927
[10,   100] loss : 0.041029
[10,   200] loss : 0.041648
[11,   100] loss : 0.040491
[11,   200] loss : 0.042656
[12,   100] loss : 0.042033
[12,   200] loss : 0.041807
[13,   100] loss : 0.041347
[13,   200] loss : 0.041447
[14,   100] loss : 0.040919
[14,   200] loss : 0.041121
[15,   100] loss : 0.042285
[15,   200] loss : 0.041246
[16,   100] loss : 0.041792
[16,   200] loss : 0.041596
[17,   100] loss : 0.040784
[17,   200] loss : 0.042972
[18,   100] loss : 0.042995
[18,   200] loss : 0.040528
[19,   100

[146,   100] loss : 0.039899
[146,   200] loss : 0.040654
[147,   100] loss : 0.039280
[147,   200] loss : 0.040938
[148,   100] loss : 0.039208
[148,   200] loss : 0.041221
[149,   100] loss : 0.039834
[149,   200] loss : 0.040039
[150,   100] loss : 0.038946
[150,   200] loss : 0.041099
[151,   100] loss : 0.039174
[151,   200] loss : 0.040880
[152,   100] loss : 0.040081
[152,   200] loss : 0.040439
[153,   100] loss : 0.039884
[153,   200] loss : 0.040409
[154,   100] loss : 0.039138
[154,   200] loss : 0.040044
[155,   100] loss : 0.040617
[155,   200] loss : 0.039837
[156,   100] loss : 0.040361
[156,   200] loss : 0.043463
[157,   100] loss : 0.040086
[157,   200] loss : 0.041519
[158,   100] loss : 0.038667
[158,   200] loss : 0.041210
[159,   100] loss : 0.040664
[159,   200] loss : 0.038721
[160,   100] loss : 0.040346
[160,   200] loss : 0.039266
[161,   100] loss : 0.039724
[161,   200] loss : 0.039906
[162,   100] loss : 0.040008
[162,   200] loss : 0.040034
[163,   100] l

[287,   200] loss : 0.037874
[288,   100] loss : 0.037312
[288,   200] loss : 0.038753
[289,   100] loss : 0.037914
[289,   200] loss : 0.039694
[290,   100] loss : 0.038070
[290,   200] loss : 0.037575
[291,   100] loss : 0.038398
[291,   200] loss : 0.041941
[292,   100] loss : 0.040295
[292,   200] loss : 0.039228
[293,   100] loss : 0.038430
[293,   200] loss : 0.038274
[294,   100] loss : 0.037725
[294,   200] loss : 0.037537
[295,   100] loss : 0.037163
[295,   200] loss : 0.037989
[296,   100] loss : 0.038177
[296,   200] loss : 0.038762
[297,   100] loss : 0.037549
[297,   200] loss : 0.038020
[298,   100] loss : 0.037624
[298,   200] loss : 0.038179
[299,   100] loss : 0.037226
[299,   200] loss : 0.037819
[300,   100] loss : 0.037341
[300,   200] loss : 0.038372
[301,   100] loss : 0.038347
[301,   200] loss : 0.039961
[302,   100] loss : 0.038704
[302,   200] loss : 0.037576
[303,   100] loss : 0.039782
[303,   200] loss : 0.036693
[304,   100] loss : 0.038292
[304,   200] l

[429,   100] loss : 0.037463
[429,   200] loss : 0.036665
[430,   100] loss : 0.036396
[430,   200] loss : 0.035235
[431,   100] loss : 0.035823
[431,   200] loss : 0.036033
[432,   100] loss : 0.036443
[432,   200] loss : 0.035073
[433,   100] loss : 0.035877
[433,   200] loss : 0.036582
[434,   100] loss : 0.035485
[434,   200] loss : 0.035989
[435,   100] loss : 0.036662
[435,   200] loss : 0.038423
[436,   100] loss : 0.036435
[436,   200] loss : 0.035515
[437,   100] loss : 0.037203
[437,   200] loss : 0.034752
[438,   100] loss : 0.035431
[438,   200] loss : 0.035838
[439,   100] loss : 0.035267
[439,   200] loss : 0.036221
[440,   100] loss : 0.036201
[440,   200] loss : 0.036492
[441,   100] loss : 0.037413
[441,   200] loss : 0.040217
[442,   100] loss : 0.036117
[442,   200] loss : 0.036268
[443,   100] loss : 0.036283
[443,   200] loss : 0.035621
[444,   100] loss : 0.035128
[444,   200] loss : 0.037292
[445,   100] loss : 0.036458
[445,   200] loss : 0.036952
[446,   100] l

## Evaluate Train data

In [280]:
n_predict = 0
n_correct = 0
for data in train_loader:
    inputs, labels = data
    outputs,_ = model(inputs)
    _, predicted = torch.max(outputs,1)

    n_predict += len(predicted)
    n_correct += (labels == predicted).sum()
    
print(f"{n_correct}/{n_predict}")
print(f"Accuracy: {n_correct/n_predict: .3f}")

<ipython-input-273-a89117153749>:30: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  a = self.log_softmax(z)


1925/2822
Accuracy:  0.682


In [285]:
model = torch.load("./model/model.pt")

## Evaluate Test data

In [286]:
n_predict = 0
n_correct = 0
for data in test_loader:
    inputs, labels = data
    outputs,_ = model(inputs)
    _, predicted = torch.max(outputs,1)

    n_predict += len(predicted)
    n_correct += (labels == predicted).sum()
    
print(f"{n_correct}/{n_predict}")
print(f"Accuracy: {n_correct/n_predict: .3f}")

<ipython-input-273-a89117153749>:30: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  a = self.log_softmax(z)


385/706
Accuracy:  0.545


## Save Model

In [259]:
if not os.path.isdir("./model") :
    os.mkdir("./model")
torch.save(model,"./model/model.pt") ## prac - 0.554 , test - 0.545

In [267]:
torch.save(model,"./model/model1.pt") ## prac - 0.566 , test - 0.545, epoch 100,layer : [256,128,64,32] , lr = 0.0005,weight_decay = 1e-5

In [278]:
torch.save(model,"./model/model2.pt") ## prac - 0.600, test - 0.520, epcoh : 500, layer :[512,256,128,64,32], lr = 0.0001, weight_decay = 1e-5